In [1]:
#importing libraries
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from math import sqrt
from scipy import stats
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, KBinsDiscretizer


In [2]:

def Get_Clean_Combined_Dataset():

    df = pd.read_csv("tcd-ml-1920-group-income-train.csv")
    df
    
    df.drop_duplicates(subset ="Instance", 
                     keep = False, inplace = True)
    df2 = pd.read_csv("tcd-ml-1920-group-income-test.csv")
    df2["Instance"] = df2["Instance"] + 991709
    
    data = pd.concat([df,df2], axis=0)
    data = data.reset_index(drop=True)
    data['Yearly Income in addition to Salary (e.g. Rental Income)'] = data['Yearly Income in addition to Salary (e.g. Rental Income)'].str.replace(' EUR', '').astype(float)
    return data

In [3]:


data_set = Get_Clean_Combined_Dataset()
print(data_set["Yearly Income in addition to Salary (e.g. Rental Income)"].max())
data_set["Total Yearly Income [EUR]"] = data_set["Total Yearly Income [EUR]"] - data_set["Yearly Income in addition to Salary (e.g. Rental Income)"]
#print(data_set["Total Yearly Income [EUR]"].sum())

c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


162285.45


In [4]:

#Split data into income set (Incuding instance so that the unlabled data can extracted later)
y = data_set[["Instance","Total Yearly Income [EUR]","Yearly Income in addition to Salary (e.g. Rental Income)"]].copy()

#And all the Feature data (Incuding instance so that the unlabled data can extracted later)
X = data_set[["Instance","Year of Record","Age"]].copy()
X.head()

,Instance,Year of Record,Age
0,1,1940.0,45
1,2,1940.0,17
2,3,1940.0,48
3,4,1940.0,42
4,5,1940.0,15


In [5]:
#Function to relpace invalid entries in numeric features with the mean walue of the particular feature
def ReplaceNan_Numeric(df,F_Name):
    average = df[F_Name].dropna().mean(axis=0)
    df[F_Name] = df[F_Name].replace(np.nan, average, inplace=False)
    return df

In [6]:
# Function to convert non-linear numeric data into 'bins' of nearby values, bins represented with onehot encoding
def KBins_Feature(df, F_Name):
    feature_array = df[F_Name].to_numpy()
    feature_array = feature_array.reshape(-1, 1)
    est = KBinsDiscretizer(n_bins=8, encode='onehot-dense', strategy='quantile')
    est.fit(feature_array)
    onehot_encoded = est.transform(feature_array)
    
    onehot_feature_df = pd.DataFrame(onehot_encoded)
    df = df.drop(F_Name,1)   #Feature Matrix
    new_df = pd.concat([df, onehot_feature_df], axis=1)
    return new_df

In [7]:

def MinMaxScale_Feature(df, F_Name):
    feature_array = df[F_Name].to_numpy()
    feature_array = feature_array.reshape(-1, 1)
    
    scaler = MinMaxScaler()
    scaled_array = scaler.fit_transform(feature_array)
    
    scaled_df = pd.DataFrame(scaled_array)
    df = df.drop(F_Name,1)   #Feature Matrix
    new_df = pd.concat([df, scaled_df], axis=1)
    return new_df

In [8]:
# Clean numeric data
X = ReplaceNan_Numeric(X,"Year of Record")
X = ReplaceNan_Numeric(X,"Age")


In [9]:
# Scale numeric data
X = MinMaxScale_Feature(X,"Year of Record")
X = MinMaxScale_Feature(X,"Age")


In [10]:
#Split the data back out to separate sets on key 'Instance'
df2_X = X.loc[X["Instance"] > 991709]
X = X.loc[X["Instance"] <= 991709]         
df2_y = y.loc[y["Instance"] > 991709]
y = y.loc[y["Instance"] <= 991709]

#Drop 'Instance' from datasets now that it is no longer needed as a key
df2_X = df2_X.drop("Instance",1)
X = X.drop("Instance",1)
y = y.drop("Instance",1)

In [11]:
# Split the test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

add_income = y_test["Yearly Income in addition to Salary (e.g. Rental Income)"].values

y_train = y_train.drop("Yearly Income in addition to Salary (e.g. Rental Income)",1)
y_test = y_test.drop("Yearly Income in addition to Salary (e.g. Rental Income)",1)


In [12]:


# Create ridgecv regression object with 4 crossvalidation folds
regr = RidgeCV(alphas=np.array([0.0001,0.001,0.01,0.1,1]),fit_intercept=True,normalize=False,cv = 4)

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)



In [13]:
length = len(add_income)
add_income = add_income.reshape(length,1)
y_pred = np.add(y_pred,add_income)


In [15]:
# Print results
print("Mean absolute error: %.2f"
      % sqrt(mean_absolute_error(y_test, y_pred)))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean absolute error: 238.14
Variance score: 0.39


In [16]:
# Make predictions on the unlabled dataset
df2_y_pred = regr.predict(df2_X)
# export data
pd.DataFrame(df2_y_pred).to_csv("Predictions.csv")